## Are there any transcripts that are expressed in all samples of one population and no samples of any other population?

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import cerberus
import pyranges as pr
import upsetplot

p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

In [2]:
config = load_config()
od = '../'

def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

In [69]:
f = expand(proc_cfg(config['lr']['kallisto']['quant']['merge_matrix_tpm_tsv'],od))[0]

meta = load_meta()
meta = meta.loc[meta.merged_run_mode==True]
sample_d = dict([(entry.cell_line_id, entry['sample']) \
                 for ind, entry in meta.iterrows()])

df = pd.read_csv(f, sep='\t')
df.columns = [d if d == 'transcript_id' else d.split('_')[0] for d in df.columns]
df.rename({'transcript_id':'tid'}, axis=1, inplace=True)
df.rename(sample_d, axis=1, inplace=True)
df.set_index('tid', inplace=True)
df['temp_merge'] = 1
df.head()

,YRI5,YRI6,YRI7,HAC1,HAC2,HAC3,HAC4,HAC5,HAC6,LWK1,...,CEU1,CEU2,CEU3,CEU4,CEU5,MPC4,YRI1,YRI2,YRI3,temp_merge
tid,,,,,,,,,,,,,,,,,,,,,
ENST00000413811.3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1
transcript_6675,0.393218,0.000000,0.552231,0.127078,0.386482,0.313976,0.498281,0.328355,0.437621,0.385320,...,0.269962,0.108892,0.172414,0.405509,0.073628,0.835420,0.214743,0.477583,0.59838,1
transcript_6676,0.956060,0.438777,1.184180,0.757060,1.551000,0.266020,0.195867,0.268813,0.351186,0.504766,...,0.519386,0.566595,0.495574,1.004550,0.540054,0.397010,0.998557,0.344694,0.18226,1
transcript_6684,0.196609,1.245200,0.631121,0.000000,0.552185,0.444888,0.394330,0.492533,0.475874,0.481651,...,0.539923,0.000000,0.822254,0.354531,0.441766,0.858465,0.214743,0.477583,0.29919,1
transcript_6687,0.000000,0.000000,0.000000,0.055554,0.000000,0.088617,0.000000,0.000000,0.056812,0.000000,...,0.000000,0.247031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1


In [70]:
# add max # samples for each pop
exp_in_pop_cols = []
for p in meta['population'].unique():
    samps = meta.loc[meta.population==p, 'sample'].tolist()
    samps = [s for s in samps if s in df.columns]
    df[f'n_exp_{p}'] = (df[samps]>0).sum(axis=1)
    df[f'exp_in_{p}'] = (df[samps]>0).sum(axis=1)>0
    exp_in_pop_cols.append(f'exp_in_{p}')

temp = meta[['sample', 'population']].groupby('population').nunique().rename({'sample':'n_samples'}, axis=1)
temp = temp.transpose()
temp.columns = [f'n_samples_{c}' for c in temp.columns]
temp['temp_merge'] = 1

In [71]:
df.reset_index(inplace=True)
df = df.merge(temp, how='left',
              on='temp_merge')
df.drop('temp_merge', axis=1, inplace=True)

In [72]:
# call things as expressed in all samples of pop or not
exp_in_all_cols = []
for p in meta['population'].unique():
    samps = meta.loc[meta.population==p, 'sample'].tolist()
    samps = [s for s in samps if s in df.columns]
    df[f'exp_in_all_{p}'] = df[f'n_exp_{p}']==df[f'n_samples_{p}']
    exp_in_all_cols.append(f'exp_in_all_{p}')


In [73]:
df.to_csv('test.tsv', sep='\t')

In [77]:
# WRONG 
df['ultra_pop_spec'] = (df[exp_in_all_cols].sum(axis=1)==1)&\
                       (df[exp_in_pop_cols].sum(axis=1)==1)
df['pop_spec'] = df[exp_in_pop_cols].sum(axis=1)==1

In [78]:
df.to_csv('test.tsv', sep='\t')

In [58]:
df.loc[df.tid=='transcript_8852']

,tid,YRI5,YRI6,YRI7,HAC1,HAC2,HAC3,HAC4,HAC5,HAC6,...,n_samples_YRI,exp_in_all_ITU,exp_in_all_PEL,exp_in_all_HAC,exp_in_all_AJI,exp_in_all_LWK,exp_in_all_YRI,exp_in_all_CEU,exp_in_all_MPC,ultra_pop_spec
49,transcript_8852,0.450956,0.0,0.0,0.201096,0.340745,0.0,0.150552,0.347292,0.200877,...,7,False,False,False,True,False,False,False,False,True


In [76]:
df.loc[df.ultra_pop_spec==True][exp_in_all_cols+['ultra_pop_spec']].head()

,exp_in_all_ITU,exp_in_all_PEL,exp_in_all_HAC,exp_in_all_AJI,exp_in_all_LWK,exp_in_all_YRI,exp_in_all_CEU,exp_in_all_MPC,ultra_pop_spec


In [84]:
ups_df = df.loc[df.ultra_pop_spec==True]

In [85]:
len(ups_df.index)

0

In [86]:
# there are none

## What populations have UPS transcripts?

In [79]:
# ups_df['ups_pop'] = df[exp_in_all_cols].idxmax(axis=1).str.split('_',expand=True)[3]

In [81]:
# ups_df[['tid', 'ups_pop']].groupby('ups_pop').nunique().rename({'tid':'n_t'}, axis=1)

## What does their expression look like?

In [82]:
# temp = ups_df.copy(deep=True)
# temp.drop([c for c in temp.columns if 'n_exp' in c], axis=1, inplace=True)
# temp.drop([c for c in temp.columns if 'n_samples' in c], axis=1, inplace=True)
# temp.drop([c for c in temp.columns if 'exp_in_all' in c], axis=1, inplace=True)
# temp.drop('ultra_pop_spec', axis=1, inplace=True)
# temp.head()

In [83]:
# # add population info
# df = df.transpose()
# df = df.merge(meta[['sample', 'population']], 
#               how='left',
#               left_index=True,
#               right_on='sample')
# df.drop('sample', axis=1, inplace=True)

## What biotypes are they?

## What strctural categories are they?